In [83]:
#Dependencias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split as tt_split 
from sklearn.preprocessing import StandardScaler as SScaler

from sklearn.linear_model import LinearRegression as Lreg
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.neural_network import MLPClassifier as MLPC

from sklearn.metrics import mean_squared_log_error as MSLE #se usa este indicador porque es lo que se pide optimizar

In [84]:
#cargando train set
df = pd.read_csv('train.csv')

In [6]:
##df.info()

In [5]:
#Diferencia entre set de entrenamiento y prueba
#Esta parte es necesaria dada la competencia, no seria un tema si fuera un caso real producto 
#de que nos encargariamos de que la distribucion tuviera las mismas caracteristicas en 
#entrenamiento y prueba

In [85]:
train_info = df.describe().iloc[1,:]

In [86]:
df_test = pd.read_csv('test.csv')

In [87]:
test_info = df_test.describe().iloc[1,:]

In [74]:
#Diferencia Absoluta
abs((train_info - test_info)/test_info).sort_values(ascending=False)

3SsnPorch        0.900149
Id               0.666438
LowQualFinSF     0.649353
PoolArea         0.581627
MiscVal          0.252354
BsmtHalfBath     0.117606
ScreenPorch      0.117406
BsmtFinSF2       0.115357
EnclosedPorch    0.094426
LotArea          0.071052
2ndFlrSF         0.064499
Fireplaces       0.054702
MoSold           0.035670
OpenPorchSF      0.034227
MasVnrArea       0.029552
BsmtUnfSF        0.023355
LotFrontage      0.021429
BsmtFullBath     0.020973
TotRmsAbvGrd     0.020769
GrLivArea        0.019796
HalfBath         0.013824
WoodDeckSF       0.011481
TotalBsmtSF      0.010813
BsmtFinSF1       0.010100
MSSubClass       0.008384
1stFlrSF         0.005268
BedroomAbvGr     0.004355
KitchenAbvGr     0.003914
OverallCond      0.003878
FullBath         0.003737
OverallQual      0.003371
YearRemodAdd     0.000606
GarageCars       0.000569
GarageArea       0.000447
GarageYrBlt      0.000397
YearBuilt        0.000046
YrSold           0.000023
SalePrice             NaN
Name: mean, 

In [11]:
#Explorando las variables

In [12]:
from pandas_profiling import ProfileReport as PR

In [82]:
prof = PR(df)
prof.to_file(output_file='train_ds.html')

prof2 = PR(df_test)
prof.to_file(output_file='test_ds.html')

NameError: name 'PR' is not defined

# Variables

In [88]:
#Variables Continuas
def get_var_con(df):
    df_con = df[['OverallQual','OverallCond','YearBuilt',\
             'TotalBsmtSF','GarageYrBlt','Fireplaces',\
             'GarageArea','FullBath','HalfBath',\
             '2ndFlrSF','GrLivArea','YearRemodAdd',\
             'MasVnrArea'
             ]]
    return df_con

In [89]:
#Funcion de Preproceso

In [90]:
#Diccionario de cambios
Variables_dict = [ \
[['Utilities'],{'AllPub':4,'NoSewr':3, 'NoSeWa':2, 'ELO':1}], \
[['ExterQual','ExterCond','BsmtQual','BsmtCond','HeatingQC',\
  'KitchenQual','FireplaceQu','GarageQual','GarageCond','PoolQC'] \
 , {'Ex':5,'Gd':4, 'TA':3, 'Fa':2, 'Po':1, np.nan : 0}], \
[['BsmtExposure'] , {'Gd':4,'Av':3, 'Mn':2, 'No':1, np.nan : 0}], \
[['BsmtFinType1','BsmtFinType2'] , {'GLQ':6,'ALQ':5, 'BLQ':4, 'Rec':3,'LwQ':2,'Unf':1,np.nan : 0}], \
[['CentralAir'],{'N':0, 'Y':1 ,np.nan : 0}], \
[['GarageFinish'] , {'Fin':3,'RFn':2, 'Unf':1, np.nan : 0}], \
[['Fence'] , {'GdPrv':4,'MnPrv':3, 'GdWo':2, 'MnWw':1, np.nan : 0}] \
]

In [91]:
def reemplazo_dict(df, x, dic):
    df[x] = df[x].map(dic).fillna(df[x])
    return df

In [92]:
def get_var_dicc(df):
    #Variables para Cambio por diccionario
    df_dict = df[['Utilities',
    'ExterQual','ExterCond','BsmtQual','BsmtCond',
    'BsmtExposure',
    'BsmtFinType1',
    'BsmtFinType2',
    'HeatingQC',
    'CentralAir',
    'KitchenQual',
    'FireplaceQu',
    'GarageFinish',
    'GarageQual',
    'GarageCond',
    'PoolQC',
    'Fence']]
    for elemento in Variables_dict:
        for columna in elemento[0]:
            df_dict = reemplazo_dict(df_dict, columna, elemento[1])  
    return df_dict

In [93]:
def AgregaOneHotEncoding(df, x):
    lista_tipos = tuple(df[x].unique())
    sub_df = pd.DataFrame(lista_tipos, columns=[x])
    dum_df = pd.get_dummies(sub_df, columns = [x], prefix = [x] )
    sub_df = sub_df.join(dum_df)
    sub_df
    
    df_final = df.merge(sub_df, how='left', on=x)
    df_final = df_final.drop(x,1)
    return df_final

In [94]:
def get_var_OHE(df):
    #Variables para OneHotEncoding
    df_OHE = df[['MSZoning',
    'Street',
    'Alley',
    'LotShape',
    'LandContour',
    'LotConfig',
    'LandSlope',
    'Neighborhood',
    'Condition1',
    'Condition2',
    'BldgType',
    'HouseStyle',
    'RoofStyle',
    'RoofMatl',
    'Exterior1st',
    'Exterior2nd',
    'MasVnrType',
    'Foundation',
    'Heating',
    'Electrical',
    'Functional',
    'GarageType',
    'PavedDrive',
    'MiscFeature',
    'SaleType',
    'SaleCondition']]
    
    for columna in  list(df_OHE.columns):
        df_OHE = AgregaOneHotEncoding(df_OHE, columna)
    return df_OHE

In [158]:
def Preprocessing(df,train_flg):
    
    df['GarageYrBlt'] = df['GarageYrBlt'].fillna(0)
    df['MasVnrArea'] = df['MasVnrArea'].fillna(0)
    df['GarageArea'] = df['GarageArea'].fillna(0)
    
        
    Scale = SScaler()

    if train_flg == 1:
        y = df['SalePrice'].values
        
    
        X1 = get_var_con(df)
        
        X2 = get_var_dicc(df)
        
        X3 = get_var_OHE(df)
 
        
        X = pd.concat([X1,X2,X3], axis=1).values

        X_out = pd.concat([X1,X2,X3], axis=1)

        X_train, X_test, y_train, y_test = tt_split(X, y, train_size= 0.67, random_state=42)



        X_train = Scale.fit_transform(X_train)

        X_test = Scale.fit_transform(X_test)

        X_train = np.nan_to_num(X_train, nan = 0)
        X_test = np.nan_to_num(X_test, nan = 0)
        
        return  X_train, X_test, y_train, y_test, X_out
    else:
        
        X1 = get_var_con(df)
        
        X2 = get_var_dicc(df)
        
        X3 = get_var_OHE(df)
 
        
        X = pd.concat([X1,X2,X3], axis=1)
        
        for element in [item for item in list(data_train.columns) if item not in list(X.columns)]:
            X[element] = 0
        
        X = X[list(data_train.columns)].values
        
        X_out = pd.concat([X1,X2,X3], axis=1)
        
        X_val = Scale.fit_transform(X)
        
        Id = df['Id'].values
        
        X_val = np.nan_to_num(X_val, nan = 0)
        
        return Id , X_val , X_out

In [96]:
#Funcion que prueba Modelos

In [118]:
def Prueba_modelo(X_train, X_test, y_train, y_test , modelo_input, nombre_modelo):
    modelo = modelo_input 
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    np.clip(a= y_pred, a_min=0, a_max=None , out=y_pred)
    error = MSLE(y_test, y_pred)
    nombre = nombre_modelo
    modelo_entrenado = modelo
    return [nombre, error, modelo]

In [109]:
#Modelos
Modelos = []

In [110]:
name = 'Linear'
model = Lreg()
Modelos.append([name,model])

In [111]:
name = 'Tree'
model = DTR()
Modelos.append([name,model])

In [112]:
name = 'RandomForest'
model = RFR()
Modelos.append([name,model])

In [113]:
name = 'MultiLayer'
model = MLPC()
Modelos.append([name,model])

In [114]:
Modelos

[['Linear',
  LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)],
 ['Tree',
  DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort='deprecated',
                        random_state=None, splitter='best')],
 ['RandomForest',
  RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                        max_depth=None, max_features='auto', max_leaf_nodes=None,
                        max_samples=None, min_impurity_decrease=0.0,
                        min_impurity_split=None, min_samples_leaf=1,
                        min_samples_split=2, min_weight_fraction_leaf=0.0,
                        n_estimators=100, n_jobs=None, oob_score=False,
                      

In [141]:
X_train, X_test, y_train, y_test, data_train = Preprocessing(df,1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [120]:
Output = []

In [121]:
for modelo in Modelos:
    Output.append(Prueba_modelo(X_train, X_test, y_train, y_test, modelo[1],modelo[0]))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [122]:
#Scaled
Output

[['Linear',
  609.9625399655984,
  LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)],
 ['Tree',
  0.042896006780183384,
  DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort='deprecated',
                        random_state=None, splitter='best')],
 ['RandomForest',
  0.022770563675996093,
  RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                        max_depth=None, max_features='auto', max_leaf_nodes=None,
                        max_samples=None, min_impurity_decrease=0.0,
                        min_impurity_split=None, min_samples_leaf=1,
                        min_samples_split=2, min_weight_fraction_leaf=0.0,
                        n

In [192]:
#Creacion de Resultado

In [159]:
Id, X_val, data_test = Preprocessing(df_test,0)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [160]:
modelo = Output[2][2]
Upload = modelo.predict(X_val)

In [161]:
Resultado = pd.DataFrame(list(zip(Id,Upload)), columns = ['Id','SalePrice'])

In [162]:
Resultado.to_csv('submission.csv', index=False)